In [1]:
# A simpler approach would be not to do this preprocessing and do replay including releases
# Avoiding hoykey, double clicks and drags detections 
class Preprocessing():
    def __init__(self, length_th, minpixels_th, dt_th, maxpixels_th): 
        # The thresholds for drag and doubleclicks detection determine the correct replay of task, how important are to calibrate them? 
        # Many of these functions make copies of dataframe, so it is not efficient. 
        # Drags
        self.length_th = length_th 
        self.minpixels_th = minpixels_th
        # Double clicks
        self.dt_th = dt_th 
        self.maxpixels_th = maxpixels_th

    def run(self, sample): 
        sample = self.replace_hotkeys(sample)                                                           # Detect hotkeys
        sample['trajectory'] = sample['trajectory'].map(lambda x: self.string2list(x))                  # Convert a list represented as a string into a actual list
        sample = self.replace_drags(sample)                                                             # Detect drag events
        actions = sample[sample['event'].map(lambda x: ('pressed' in x) or ('Scroll.' in x))].copy()    # Select pressed and scroll events
        actions = self.replace_doubleclicks(actions)                                                    # Detect double clicks
        actions['event'] = actions['event'].map(lambda x: x.replace('pressed ',''))       # Remove pressed string
        delays = actions['timestamp'][1:].values - actions['timestamp'][0:-1].values      # Calculating delays
        delays = np.append(delays,0.0)                                                
        actions['delay'] = delays
        actions = actions.reset_index(drop=True)                                          # Reset dataframe index
        return actions
    
    def replace_hotkeys(self, sample):
        """ Replace rows in the sample by hotkey events """
        # The key insight: consecutive pressed events form a hotkey. 
        # And the number of releases (di) between groups of consective pressed events determine the keys that are keeping press (basepressed). 
        samplecopy = sample.copy()
        for ix, ixN in self.find_hotkeys(samplecopy):
            ixs_pressed = sample[ix:ixN].index[sample['event'][ix:ixN].map(lambda x: 'pressed' in x)]
            pgroups = self.group_consecutive(ixs_pressed)  # Groups of consecutive position indexes of pressed events 
            keep_ixs = []
            basepressed = []
            for i in range(len(pgroups)): 
                pgroup = basepressed + pgroups[i]
                newevent = 'pressed ' + samplecopy.loc[pgroup[0], 'event'].replace('pressed ', '')
                for ki in pgroup[1:]: 
                    newevent += '+' + samplecopy.loc[ki, 'event'].replace('pressed ', '')
                samplecopy.loc[pgroup[-1], 'event'] = newevent
                keep_ixs.append(pgroup[-1])
                if i+1 < len(pgroups):
                    di = pgroups[i+1][0] - pgroup[-1] - 1
                    basepressed = pgroup[:-di]

            # Remove rows
            remove_ixs = [i for i in range(ix,ixN+1)]
            [remove_ixs.remove(i) for i in keep_ixs]
            samplecopy = samplecopy.drop(remove_ixs)
        return samplecopy 
    
    def group_consecutive(self, numbers):
        """ Making groups of consecutive numbers """ # numbers list cannot be empty by design.
        groups = []
        current_group = [numbers[0]]
        for i in range(1, len(numbers)):
            if numbers[i] == numbers[i-1] + 1:
                current_group.append(numbers[i])
            else:
                groups.append(current_group)
                current_group = [numbers[i]]
        groups.append(current_group)
        return groups
    
    def find_hotkeys(self, sample):
        ixs_pressed = sample.index[sample['event'].map(lambda x: 'pressed' in x)].tolist()
        hotkeys_indexes = []
        while len(ixs_pressed) >= 2:  # To not include the end Key.esc
            ix = ixs_pressed[0]
            released1 = sample['event'][ix].replace('pressed', 'released')
            ixN = sample[ix:].index[sample['event'][ix:] == released1][0]
            if ixN - ix >= 3:  # 3 para evitar que al escribir aparezcan hotkeys. Cuidado!! es una condicion debil
                hotkeys_indexes.append((ix, ixN))
            for i in range(ix,ixN):
                try: 
                    ixs_pressed.remove(i)
                except:
                    pass
        return hotkeys_indexes

    def string2list(self, string): 
        """ Convert the trajectory string into an actual python list"""
        if isinstance(string, str):
            aux1 = string.strip('][').split('), (')
            if aux1[0] != '':
                aux2 = [x.strip(')(').split(', ') for x in aux1]
                trajectory = [(float(time), int(px), int(py)) for time,px,py in aux2]
            else:
                trajectory = []
        else:
            trajectory = []
        return trajectory

    def replace_drags(self, sample):
        """ Transform the dataframe to encode drag events"""
        samplecopy = sample.copy()
        samplecopy['drag2px'] = len(samplecopy)*[None]
        samplecopy['drag2py'] = len(samplecopy)*[None]
        for ix in self.find_drags(sample):
            samplecopy.loc[ix, 'event'] = 'pressed Button.left.drag'
            samplecopy.loc[ix, 'drag2px'] = sample.loc[ix+1, 'px']
            samplecopy.loc[ix, 'drag2py'] = sample.loc[ix+1, 'py']
            samplecopy.at[ix, 'trajectory'] = sample.loc[ix+1, 'trajectory']
        return samplecopy
    
    def find_drags(self, sample):
        rBleft_indexes = sample.index[sample['event'].map(lambda x: 'released Button.left' in x)]
        drags_indexes = []
        for ix in rBleft_indexes:
            length = len(sample.loc[ix,'trajectory'])
            p1 = (sample.loc[ix-1,'px'],sample.loc[ix-1,'py'])
            p2 = (sample.loc[ix,'px'], sample.loc[ix,'py'])
            if (length >= self.length_th) and ((abs(p1[0]-p2[0]) >= self.minpixels_th) or (abs(p1[1] - p2[1]) >= self.minpixels_th)): 
                drags_indexes.append(ix-1)
        return drags_indexes
    
    def replace_doubleclicks(self, sample):
        """ Transform the dataframe to encode double events """
        samplecopy = sample.copy()
        for ix0,ix1 in self.find_doubleclicks(sample):
            samplecopy.loc[ix0, 'event'] = 'pressed Button.left.double'
            samplecopy = samplecopy.drop([ix1])
        return samplecopy

    def find_doubleclicks(self, sample):
        pBleft_indexes = sample.index[sample['event'].map(lambda x: 'pressed Button.left' in x)].tolist()
        dclicks_indexes = []
        while len(pBleft_indexes) > 1:
            ix0, ix1 = pBleft_indexes[0], pBleft_indexes[1]
            dt = sample.loc[ix1, 'timestamp'] - sample.loc[ix0, 'timestamp']
            p1 = (sample.loc[ix0, 'px'], sample.loc[ix0, 'py'])
            p2 = (sample.loc[ix1, 'px'], sample.loc[ix1, 'py'])
            if (dt <= self.dt_th) and (abs(p1[0]-p2[0]) <= self.maxpixels_th) and (abs(p1[1] - p2[1]) <= self.maxpixels_th): 
                dclicks_indexes.append((ix0, ix1))
                pBleft_indexes.pop(1)   
            pBleft_indexes.pop(0)
        return dclicks_indexes

from glob import glob 
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

ix_sample = 0
sample_paths = glob('../data/*/*.csv')
[print(x) for x in sample_paths]
sample_path = sample_paths[ix_sample]
print('usamos este:',sample_path)
sample = pd.read_csv(sample_path, index_col=0)

processing = Preprocessing(length_th = 5, minpixels_th = 1, dt_th = 0.22, maxpixels_th = 0)
print(processing.find_hotkeys(sample))
processing.replace_hotkeys(sample)[20:80]
processing.run(sample)[10:70]

../data/sample1/raw_pcdata_2024-03-26_07-44-31.csv
usamos este: ../data/sample1/raw_pcdata_2024-03-26_07-44-31.csv
[(26, 29), (32, 61), (66, 79), (84, 87), (88, 107), (114, 119), (126, 131), (134, 137), (140, 171)]


,timestamp,img_path,px,py,event,trajectory,drag2px,drag2py,delay
10,12.184269,data/sample1/screen0000000022_12.1377212439983...,583,394,Button.left,[],None,None,0.646586
11,12.830855,data/sample1/screen0000000024_12.7888775140017...,592,398,Button.left.drag,"[(12.88485939700331, 592, 399), (12.8849724469...",473,394,1.703042
12,14.533897,data/sample1/screen0000000027_14.4921956000034...,473,394,Key.ctrl+c,[],None,None,1.541346
13,16.075243,data/sample1/screen0000000030_16.0344271590001...,513,563,Button.left,[],None,None,2.787937
14,18.863181,data/sample1/screen0000000033_18.8342164079949...,516,564,Key.ctrl+v,[],None,None,0.749600
15,19.612781,data/sample1/screen0000000035_19.5784164990036...,516,564,Key.ctrl+Key.enter,[],None,None,2.683138
16,22.295919,data/sample1/screen0000000038_22.2619081009906...,516,564,Key.ctrl+Key.shift+V,[],None,None,0.912035
17,23.207955,data/sample1/screen0000000040_23.1774052480031...,516,564,Key.ctrl+Key.shift+Key.enter,[],None,None,0.660232
18,23.868186,data/sample1/screen0000000043_23.8282109610008...,516,564,Key.ctrl+Key.enter,[],None,None,1.761996
19,25.630182,data/sample1/screen0000000046_25.5815595369931...,516,564,Key.ctrl+Key.shift+V,[],None,None,0.921243


In [ ]:
### Version 2 
    def replace_hotkeys(self, sample):
        samplecopy = sample.copy()
        for ix, ixN in self.find_hotkeys(samplecopy):
            ixs_pressed = sample[ix:ixN].index[sample['event'][ix:ixN].map(lambda x: 'pressed' in x)]
            newevent = 'pressed ' + samplecopy.loc[ixs_pressed[0], 'event'].replace('pressed ', '')
            # event1 + ... + eventN
            if self.are_pressedkeys_consecutive(ixs_pressed.tolist()): 
                for ix_pressed in ixs_pressed[1:]: 
                    newevent += '+' + samplecopy.loc[ix_pressed, 'event'].replace('pressed ', '')
                samplecopy.loc[ixs_pressed[-1], 'event'] = newevent
                remove_ixs = ixs_pressed[:-1].tolist() + sample[ix:ixN+1].index[sample['event'][ix:ixN+1].map(lambda x: 'released' in x)].tolist()
                samplecopy = samplecopy.drop(remove_ixs)
            # event1 + {event2 + ... + eventN}
            elif self.are_sequentialhotkeys(ixs_pressed[1:].tolist()):
                for ix_pressed in ixs_pressed[1:]:
                    samplecopy.loc[ix_pressed, 'event'] = newevent + '+' + samplecopy.loc[ix_pressed, 'event'].replace('pressed ', '')
                    samplecopy = samplecopy.drop([ix_pressed+1])
                samplecopy = samplecopy.drop([ixs_pressed[0], ixN])
            # Other cases are not include: like event1 + event2 + {event3 + ... + eventN}
            else:
                # what to do? Nothing. 
                pass
        return samplecopy 
    
    def are_sequentialhotkeys(self, numbers_sorted):   
        for i in range(len(numbers_sorted) - 1):
            if numbers_sorted[i] + 2 != numbers_sorted[i + 1]:
                return False
        return True
    
    def are_pressedkeys_consecutive(self, numbers_sorted):   
        for i in range(len(numbers_sorted) - 1):
            if numbers_sorted[i] + 1 != numbers_sorted[i + 1]:
                return False
        return True
    
    def find_hotkeys(self, sample):
        ixs_pressed = sample.index[sample['event'].map(lambda x: 'pressed' in x)].tolist()
        hotkeys_indexes = []
        while len(ixs_pressed) >= 2:  # To not include the end Key.esc
            ix = ixs_pressed[0]
            released1 = sample['event'][ix].replace('pressed', 'released')
            ixN = sample[ix:].index[sample['event'][ix:] == released1][0]
            if ixN - ix >= 3:  # 3 para evitar que al escribir aparezcan hotkeys. Cuidado!! es una condicion debil
                hotkeys_indexes.append((ix, ixN))
            for i in range(ix,ixN):
                try: 
                    ixs_pressed.remove(i)
                except:
                    pass
        return hotkeys_indexes

In [ ]:
### Version 1
